In [3]:
%%writefile preproccess.py

import logging
import os
import subprocess
import pandas as pd
import yaml
import datetime
import gc
import re

# File reading #

def read_config_file(filepath):
    with open(filepath,"r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern=char+ '{2,}'
    string=re.sub(pattern,char,string)
    return string

def col_header_val(df,table_config):
#replace whitespaces in the columm and standarize col names
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]',_ ,regex=True)
    df.columns = list(map(lambda x: x.strip('='),list(df.columns)))
    df.columns = list (map(lambda x: replacer(x,'_'),list(df.columns)))
    expected_col = list (map(lambda x: x.lower(),table_config['columns']))
    expected_col.sort()
    df =df.reindex(sorted(df.columns),axis=1)

    if len(df.columns)== len(expected_col) and list(expected_col)==list(df.columns):
        print("column names and len validation passed")
        return 1
    else:
        print("column names and len validation failed")
        mismatched_col_file=list(set(df.columns).difference(expected_col))
        print("the following file columns are not in the yaml file",mismatched_col_file)
        missing_yaml_file=list(set(expected_col).difference(df.columns))
        print("the following yaml columns are not in the file uploader",missing_yaml_file)
        logging.info(f'df columns: (df.columns)')
        logging.info(f'expected columns: (expected_col)')
        return 0



Overwriting preproccess.py


In [5]:
import yaml
a_yaml_file = open("config.yaml")
config_data = yaml.load(a_yaml_file, Loader=yaml.FullLoader)

print(config_data["file_type"])

csv


In [6]:
config_data

{'file_type': 'csv',
 'dataset_name': 'nyt_comments',
 'file_name': 'nyt_comments',
 'table_name': 'nyt',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_loading_rows': 1,
 'columns': ['commentID',
  'status',
  'commentSequence',
  'userID',
  'userDisplayName',
  'userLocation',
  'userTitle',
  'commentBody',
  'createDate',
  'updateDate',
  'approveDate',
  'recommendations',
  'replyCount',
  'editorsSelection',
  'parentID',
  'parentUserDisplayName',
  'depth',
  'commentType',
  'trusted',
  'recommendedFlag',
  'permID',
  'isAnonymous',
  'articleID']}

In [8]:
import dask.dataframe as dd

df = dd.read_csv('nyt_comments.csv',delimiter=',')
df.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...


## Reading the data using Dask library ( better for larger size data), but using dask while reading using the coonfig yaml  file always gave an error, so i decided to use pandas and read the data using chinks that will be appended

In [9]:
import pandas as pd
file_type=config_data['file_type']
source_file=config_data['file_name']+ f'.{file_type}'
df_chunk=pd.read_csv(source_file,config_data['inbound_delimiter'],chunksize=100000)

chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
df_final = pd.concat(chunk_list)

C:\Users\M-ODE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df_final.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
